<a href="https://colab.research.google.com/github/michellepace/anthropic-notes/blob/main/notes01_api_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size="6"><b>Notes — API Fundementals Anthropic</b></font>

<b>Quick Reference</b>

---

Notes taken from Anthropic Github course <a href="https://github.com/anthropics/courses/tree/master/anthropic_api_fundamentals" target="_blank">API Fundementals</a>.

Written in short form, things that stood out and for quick reference later.

**Contents**

1. <a href="#id-setup">Setup</a>
1. <a href="#id-message-parameter">Message Parameter</a>
1. <a href="#id-measure-models">Measure Models</a>
1. <a href="#id-model-parameters">Model Parameters</a>
1. <a href="#id-image-helper">Image Helper</a>
1. <a href="#id-streaming">Streaming</a>

<a name="id-setup"></a>
# **SETUP**

- Libraries (VSCode)

In [ ]:
# import os
# from anthropic import Anthropic, APIError

# # Working with images
# from IPython.display import Image, display
# import base64
# import httpx

- Libraries (Google Colab)

In [ ]:
# Install first
try:
    !pip install --upgrade-strategy only-if-needed --quiet anthropic
    print("Success! Libraries installed.")
except Exception as e:
    print(f"Installation error occured: {str(e)}")

# Import all
try:
    from google.colab import userdata   # To access your Colab Secret: ANTHROPIC_API_KEY
    from anthropic import Anthropic, APIError
    from IPython.display import Image, display # Images
    import base64 # Images
    import httpx # Images
    print("Success! Libraries now imported too.")
except Exception as e:
    print(f"Importantion error occured: {str(e)}")

- Models

In [ ]:
HAIKU_3 = 'claude-3-haiku-20240307'
HAIKU = 'claude-3-5-haiku-latest'
SONNET = 'claude-3-5-sonnet-latest'
OPUS = 'claude-3-opus-latest'

default_model = HAIKU_3 # can change later

print(f'Notebook default model is:\n • {default_model}')

- Set Anthropic API Key (VSCode)

In [ ]:
# ANTHROPIC_CLIENT = Anthropic(
#     api_key = os.environ.get('ANTHROPIC_API_KEY')
# )

- Set Anthropic API Key (Colab)

In [ ]:
#@title API Key in Colab⭐ { display-mode: "form" }
#@markdown **Step 1:** Create an [Anthropic API key](https://console.anthropic.com/settings/keys).<br>
#@markdown **Step 2:** Then run this cell by clicking the 'play' icon just under the title.<br>
#@markdown **Step 3:** you will be guided to setup a Colab secret if you don't have one.<br>
#@markdown


anthropic_api_secret_name = 'ANTHROPIC_API_KEY'  # @param {type: "string"}

def test_anthropic_connection(anthropic_client: Anthropic) -> None:
    my_test_prompt = "Hello Claude. Are you there?!"

    try:
        message = anthropic_client.messages.create(
            model=default_model,
            max_tokens=20,
            messages=[{"role": "user", "content": my_test_prompt}]
        )
        print("Success!\nYour Anthropic API key works.")
        print(f"• PROMPT CLAUDE: {my_test_prompt}")
        print(f"• CLAUDE REPLIED: {message.content[0].text}✅")

    except Exception as e:
        print(f"Connection test failed: {str(e)} 🛑")
        raise KeyboardInterrupt("Stopping execution")


def validate_anthropic_api_key_format(api_key):
    if not api_key.startswith('sk-'):
        raise ValueError("Anthropic API keys start with \"sk-\"")
    if ' ' in api_key:
        raise ValueError("Anthropic API keys don't have white spaces.")
    if len(api_key) <= 100:
        raise ValueError("Anthropic API keys are longer than 100 characters.")


def get_anthropic_api_key(secret_name):
    try:
        api_key = userdata.get(secret_name)
        validate_anthropic_api_key_format(api_key)
        print('Success!')
        print(f'Your Colab secret was found: "{secret_name}"')
        print(f'• If it holds a valid API Key, we can connect to Claude.\n')
        return api_key

    except userdata.SecretNotFoundError:
        print(f'🛑Error: Did not find your Colab secret: "{secret_name}".')
        print(f' To fix:')
        print(f' 1. Click the "key" icon on the left')
        print(f' 2. Add new secret with name "{secret_name}"')
        print(f' 3. Set value to an Anthropic API key')
        print(f' 4. Rerun this block and follow next instructions')
        print(f' About Colab secrets: https://bit.ly/4cad0v7 \n')
        raise
    except userdata.NotebookAccessError:
        print(f'🛑Error: Notebook access denied for Colab secret "{secret_name}"')
        print(f' To fix:')
        print(f' 1. Rerun this block and click "Grant access"')
        print(f' About Colab secrets: https://bit.ly/4cad0v7')
        raise
    except ValueError as ve:
        print(f'🛑Error: Invalid format, {str(ve)}')
        print(f' To fix:')
        print(f' 1. Click the "key" icon on the left of this Notebook')
        print(f' 2. Delete "{anthropic_api_secret_name}"')
        print(f' 4. Rerun this block and follow next instructions')
        raise
    except Exception as e:
        print(f'🛑Error: An unexpected one.')
        print(f' Please check:')
        print(f' 1. "{secret_name}" secret exists in Colab (click "key" icon on the left)')
        print(f' 2. Secret value is a valid Anthropic API key')
        print(f' Get API key: https://console.anthropic.com/settings/keys')
        print(f' About Colab secrets: https://bit.ly/4cad0v7')
        raise


# Usage
MY_ANTHROPIC_API_KEY = get_anthropic_api_key(anthropic_api_secret_name)

ANTHROPIC_CLIENT = Anthropic(
    api_key=MY_ANTHROPIC_API_KEY,
    max_retries=2,
    timeout=10, # Timeout of the retry
)

test_anthropic_connection(ANTHROPIC_CLIENT)

<a name="id-message-parameter"></a>
# **MESSAGES PARAM**

Messages parameter is a list of one or more dictionaries, where each dictionary has two keys:

`role`:
- Either 'user" or "assistant" (must alternate)

`content`:
- Can be a string (will be treated as single text content block).
- Can be a list of content dictionaries, each with a "type" (e.g., "text" or "image") and the corresponding content.

In [ ]:
claude_response = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=100,
    messages=[
        {'role': 'user', 'content': 'Hello Claude, today is Monday.'},
        {'role': 'assistant', 'content': 'Okay, got it. Today is Monday.'},
        {'role': 'user', 'content': 'What day is it?'},
    ]
)

print(claude_response.content[0].text)

## Images

Anthropic Tip:
> Just as with document-query placement, Claude works best when images come before text. Images placed after text or interpolated with text will still perform well, but if your use case allows it, we recommend an image-then-text structure. From Anthropic docs [here](https://docs.anthropic.com/en/docs/build-with-claude/vision#prompt-examples).

* Get the image setup:

In [ ]:
from IPython.display import Image, display
import base64
import httpx

dog_image_url = 'https://michellepace.github.io/anthropic-notes/images_mine/test_dog.jpg'
image1_media_type = "image/jpeg"
image1_data = base64.standard_b64encode(httpx.get(dog_image_url).content).decode("utf-8")

display(Image(url=dog_image_url))

* Use the image in a conversation

In [ ]:
claude_response = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=100,
    messages=[
        {'role': 'user', 'content': 'Hello Claude, today is Monday.'},
        {'role': 'assistant', 'content': 'Okay, got it. Today is Monday.'},
        {'role': 'user', 'content': [
            {'type': 'image', 'source': {'type': 'base64', 'media_type': image1_media_type, 'data': image1_data}},
            {'type': 'text', 'text': 'Is the day in this image the same?'}
        ]}
    ]
)

display(Image(url=dog_image_url))
print(claude_response.content[0].text)

## Prefilling

Same as above, but we prefill words into Claude's mouth on the last message in the conversation:

In [ ]:
prefil_words = 'In the image I see that the day is:'

claude_response = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=100,
    messages=[
        {'role': 'user', 'content': 'Hello Claude, today is Monday.'},
        {'role': 'assistant', 'content': 'Okay, got it. Today is Monday.'},
        {'role': 'user', 'content': [
            {'type': 'image', 'source': {'type': 'base64', 'media_type': image1_media_type, 'data': image1_data}},
            {'type': 'text', 'text': 'Is the day in this image the same?'}
        ]},
        {'role': 'assistant', 'content': prefil_words}
    ]
)

print(f'Words I put in Claude\'s mouth:\n• "{prefil_words}"\n')
print(f'Claude Response:\n• "{claude_response.content[0].text}"\n')
print(f'Showing them together:\n• "{prefil_words + claude_response.content[0].text}"')

## Shot Examples

### plain txt

In [ ]:
tweets = [
    'I love pickles',
    'I hate pickles',
    'I eat pickles on Tuesdays',
    'Just tried the new spicy pickles from @PickleCo, and my taste buds are doing a happy dance! 🌶️🥒 #pickleslove #spicyfood',
]

prompt = f"""For each tweet in this list {tweets}, classify its sentiment as positive, negative, or neutral.
For each tweet, return the classification in the format: "Tweet: [tweet text] | Sentiment: [sentiment]"

Examples:
Tweet: I could not live without pickles!! | Sentiment: positive
Tweet: If I just look at pickles I die. | Sentiment: negative
Tweet: She puts chillies on my sandwich. | Sentiment: neutral
"""

reponse = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=1000,
    temperature=1,
    messages=[{
        "role": "user",
        "content": [{"type": "text", "text": prompt}]
    }]
)
print(reponse.content[0].text)

### Claude loves XML

Why use XML tags?
- **Clarity**: Clearly separate different parts of your prompt and ensure your prompt is well structured.
- **Accuracy**: Reduce errors caused by Claude misinterpreting parts of your prompt.
- **Flexibility**: Easily find, add, remove, or modify parts of your prompt without rewriting everything.
- **Parseability**: Having Claude use XML tags in its output makes it easier to extract specific parts of its response by post-processing.

See Docs: <a href="https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags" target="_blank">Use XML tags to structure your prompts</a>

In [ ]:
tweets = [
    'I love pickles',
    'I hate pickles',
    'I will eat pickles on Tuesday only',
    'Just tried the new spicy pickles, happy dance! 🌶️🥒 #pickleslove #spicyfood',
]

prompt = f"""For each tweet in this list {tweets}, classify its sentiment as positive, negative, or neutral. Format each response using the exact XML structure shown in the examples below.

Examples:

<tweet>
    <text>If I look at chillies I die.</text>
    <sentiment>negative</sentiment>
</tweet>

<tweet>
    <text>My wife puts chillies on my sandwich.</text>
    <sentiment>neutral</sentiment>
</tweet>
"""

reponse = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=1000,
    temperature=1,
    messages=[{"role": "user", "content": [{"type": "text", "text": prompt}]}]
)
print(reponse.content[0].text)

<a name="id-measure-models"></a>
# **MEASURE MODELS**

> **GENERAL**

Major factors to consider:

* The model's capabilities — smartness (do first)
* The model's latency — speed (effected by whose serving the model)
* The model's cost — money (watch out for hidden costs)
* Security (see AI Engineering, Chapter 5: <a href="https://learning.oreilly.com/library/view/ai-engineering/9781098166298/ch05.html#ch05a_defensive_prompt_engineering_1730156991196256" target="_blank">Defensive Prompt Engineering</a>)

Simple comparision of speed, capability, cost
* See article: <a href="https://ailearnlog.com/pushing-aside-the-bench-for-the-mark/" target="_blank">Pushing Aside the Bench for the Mark: Choosing an LLM</a>
* See supporting code: <a href="https://colab.research.google.com/github/michellepace/anthropic-model-compare/blob/main/Anthropic_Model_Compare_(simple).ipynb" target="_blank">Anthropic_Model_Compare_(simple).ipynb</a>

Notes on Speed:
* Units: commonly seconds/token, but I like tokens/second.
* Experienced by product: Speed = (output tokens ÷ execution duration).
* Also effected by where you are getting the model served from (eg AWS or  Anthropic themselves via their API), your network, and current congestion!
* Use sampling (don't just measure the prompt once)

Notes on Capaibility
* Easy to measure if it is a deterministic answer (eg math solution)
* Likely to be non-deterministic: lots of different ways to evaluate
* Either way, use sampling, run on different days - just like speed - see article <a href="https://ailearnlog.com/pushing-aside-the-bench-for-the-mark/#appendix-i" target="_blank">Appendix I</a>.

> **TIPS ANTHROPIC**

Start Haiku light:
> When experimenting, we often recommend starting with the Haiku model. Haiku is a lightweight and fast model that can serve as an excellent starting point for many applications. Its speed and cost-effectiveness make it an attractive option for initial experimentation and prototyping. In many use cases, Haiku proves to be perfectly capable of generating high-quality responses that meet the needs of the application. By starting with Haiku, you can quickly iterate on your application, test different prompts and configurations, and gauge the model's performance without incurring significant costs or latency. If you are unhappy with the responses, it's easy to "upgrade" to a model like Claude 3.5 Sonnet.

❗❗❗ VERY NB — upgrade / switch
> As you develop and refine your application, it's essential to set up a comprehensive suite of evaluations specific to your use case and prompts. These evaluations will serve as a benchmark to measure the performance of your chosen model and help you make informed decisions about potential upgrades.

> By establishing a rigorous evaluation framework, you can objectively compare the performance of different models across your specific use case. This empirical evidence will guide your decision-making process and ensure that you select the model that best aligns with your application's needs.

> **CLAUDE TABLE**

Anthropic comparsion table as of Feb 2025, see latest <a href="https://docs.anthropic.com/en/docs/about-claude/models#model-comparison-table" target="_blank">here</a>.

| What | Claude 3.5 Sonnet | Claude 3.5 Haiku | Claude 3 Opus | Claude 3 Haiku |
|:--- |:--- |:--- |:--- |:--- |
| **Description** | Our most intelligent model | Our fastest model | Powerful model for highly complex tasks | Fastest and most compact model for near-instant responsiveness |
| **Strengths** | Highest level of intelligence and capability | Intelligence at blazing speeds | Top-level intelligence, fluency, and understanding | Quick and accurate targeted performance |
| **Multilingual** | Yes | Yes | Yes | Yes |
| **Vision** | Yes | No | Yes | Yes |
| **Message Batches API** | Yes | Yes | Yes | Yes |
| **Comparative latency** | Fast | Fastest | Moderately fast | Fastest |
| **Context window** | 200k tokens | 200k tokens | 200k tokens  | 200k tokens |
| **Max output** | 8192 tokens | 8192 tokens | 4096 tokens | 4096 tokens  |
| **Cost (Input / Output per )** | $3.00 / $15.00 | $0.80 / $4.00 | $15.00 / $75.00 | $0.25 / $1.25 |
| **Training data cut-off** | Apr 2024 | July 2024 | Aug 2023 | Aug 2023 |

<a name="id-model-parameters"></a>
# **MODEL PARAMS**

## Basics (required)

* `model`
* `max_tokens`: does a hard cut
* `messages`: (section: Messages Paramger)
* `temperature`: (not required b/c defaults to 1)

**Gets set like so:**

```python
reponse = ANTHROPIC_CLIENT.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens= ... etc.
)
```

## max_token

Contain costs and latency, truncaptes even if response is not finished.

I am unsure if it guides response in the same as "Target length: 40-50 words"

* `max_tokens`: hard cut in action

In [ ]:
response = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=5,
    messages=[
        {'role': 'user', 'content': 'Write a poem'},
    ]
)
print(f'Response was:\n "{response.content[0].text}"\n')
print(f'Stop reason was:\n {response.stop_reason}\n')
print(f'Output tokens was:\n {response.usage.output_tokens}\n')
print('Michelle — natural endings stop reason is: "end_turn"')

## stop_sequence
-  text strings that cause Claude to immediately stop generating when encountered
- The model will stop before generating the actual stop sequence itself
- stop reason will be: "stop_sequence"
- Useful for: ??

In [ ]:
def generate_random_letters_3_times():
    for i in range(3):
        response = ANTHROPIC_CLIENT.messages.create(
            model=default_model,
            max_tokens=500,
            messages=[{"role": "user", "content": "generate a poem"}],
            stop_sequences=["b", "c"]
        )
        print(f"Response {i+1} stopped because {response.stop_reason}.  The stop sequence was {response.stop_sequence}")


# Usage:
generate_random_letters_3_times()

## Sample: top_p

A model constructs its outputs through a process known as sampling. There's many different sample strategies: all aim to nudge models toward responses with specific attributes.

**Why care about sampling?**
- The right strategy helps generate responses suitable for application (eg creative vs predictable).
- Generate responses that follow certain formats and constraints.

**Common strategies**
- `temperature`: changes fatness of next token probability distribution. That is,adjusts the probability distribution of ALL possible tokens.
- `top_k`: cuts off the long tail (zeros out the probabilities for anything below the k’th token)... but how do you know what k to use?
- `top_p`: makes it easier, cuts off by CDF - so dynamic. So if you set 0.8, it will cut off the 20% of tokens that fall out the CDF. ThT is, dynamically selects a subset of tokens based on cumulative probability

**When to use `top_p` vs `temp`** — I don't know! ❓❓

I am unsure, but Anthropic docs says use one or the other. Presumably because of interaction and so unpredictability. But that doesn't makes sense mathematically? Use Cases for `top_p` acccording to AI Engineering, Chip:
- The scope of valid responses should vary based on context
- You need contextually appropriate outputs
- You need contextually appropriate responses but don't want to manually adjust parameters for each prompt

<a href="https://michellepace.github.io/anthropic-notes/images_mine/photo_top_p.jpg" target="_blank">
  <img src="https://michellepace.github.io/anthropic-notes/images_mine/photo_top_p.jpg" width="400px">
</a>

## Sample: temp
When generating text, Claude **predicts the probability distribution of the next token**. The temperature parameter is used to manipulate this probability distribution before sampling the next token.
- **LOW**: distribution more peaked on "safe" choices (math questions!)
- **HIGH**: distribution fattens out (not centered on only highly probable generic choices - ie creative).
- Range [0, 1] and default: 1

<img src="https://michellepace.github.io/anthropic-notes/images_mine/temperature.jpg"/>

* Demo temprature:

In [ ]:
def demonstrate_temperature(turns):
    temperatures = [0, 1]
    for temperature in temperatures:
        print(f"Prompting Claude {turns} times with temperature of {temperature}")
        print("================")
        for i in range(turns):
            response = ANTHROPIC_CLIENT.messages.create(
                model=default_model,
                max_tokens=100,
                messages=[{"role": "user", "content": "What colour is the most magical elephant? Only respond with the colour."}],
                temperature=temperature
            )
            print(f"Response {i+1}: {response.content[0].text}")
        print()

# Usage:
demonstrate_temperature(4)

## system prompt

Litrally just the text that gets prefixed to the top of the prompt. Sets the stage for the conversation: high-level instructions, defining its role, or providing background information that should inform its responses.

Anthropic says use it for: tone, context role.
- NOT: Detailed instructions, external input content (such as documents)
- NOT: examples, they should go inside the first `User` turn for better results.

In [ ]:
def demonstrate_system_prompt(turns):
    top_p_values = [0.1, 0.7, 0.95]
    for top_p in top_p_values:
        print(f"Prompting Claude {turns} times with top_p of {top_p}")
        print("================")
        for i in range(turns):
            response = ANTHROPIC_CLIENT.messages.create(
                model=default_model,
                max_tokens=20,
                system="You are an expert BMW salesperson. All magical elephants come in BMW car colours.",
                messages=[{"role": "user", "content": "What colour is the most magical elephant? Only respond with the colour."}],
                top_p=top_p,
                # temperature=top_p,
            )
            print(f"Response {i+1}: {response.content[0].text}")
        print()

# Usage:
demonstrate_system_prompt(8)

<a name="id-image-helper"></a>
# **IMAGE HELPER**

Cleaner way to work with images. Also for messages: nice to extract from API call.

## Helper

In [ ]:
import base64
import mimetypes
import httpx

def create_image_message(image_url_or_path: str) -> dict:
    """Create an image message block for Anthropic API call.
    Args: URL or local file path to image
    Returns: Image message block for Anthropic API call
    """
    if image_url_or_path.startswith('http'):
        response = httpx.get(image_url_or_path)
        binary_data = response.content
        mime_type, _ = mimetypes.guess_type(image_url_or_path)
    else:
        with open(image_url_or_path, "rb") as image_file:
            binary_data = image_file.read()
        mime_type, _ = mimetypes.guess_type(image_url_or_path)

    base64_string = base64.b64encode(binary_data).decode('utf-8')

    return {
        "type": "image",
        "source": {"type": "base64", "media_type": mime_type, "data": base64_string}
    }

## messy msgs
Usage like before but with function (see beginning of Notebook)

In [ ]:
# Claude call
claude_response = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=100,
    messages=[
        {'role': 'user', 'content': 'Hello Claude, today is Monday.'},
        {'role': 'assistant', 'content': 'Okay, got it. Today is Monday.'},
        {'role': 'user', 'content': [
            create_image_message(dog_image_url),
            {'type': 'text', 'text': 'Is the day in this image the same?'}
        ]}
    ]
)

# Show
display(Image(url=dog_image_url))
print(claude_response.content[0].text)

## nice msgs
Usage but more organised and clean:

In [ ]:
# Show image
pig_image_url = 'https://michellepace.github.io/anthropic-notes/images_mine/test_pig.jpg'
display(Image(url=pig_image_url))

# Get messages organised (outside API call)
messages = [
    {'role': 'user', 'content': 'Hello Claude, today is Monday.'},
    {'role': 'assistant', 'content': 'Okay, got it. Today is Monday.'},
    {'role': 'user', 'content': [
        create_image_message(pig_image_url),
        {'type': 'text', 'text': 'Is the day in this image the same?'}
    ]}
]

# Make call to Claude
claude_response = ANTHROPIC_CLIENT.messages.create(
    model=default_model,
    max_tokens=100,
    messages=messages
)

# Show Claude response
print(claude_response.content[0].text)

<a name="id-streaming"></a>
# **STREAMING**

* Improve user experience: see text as it's getting generated
* You care about reducing "time to first token" (ie time until you see something)
* Skip for now, come back later: <a href="https://github.com/anthropics/courses/blob/master/anthropic_api_fundamentals/05_Streaming.ipynb" target="_blank">05_Streaming.ipynb</a>